In [35]:
#carregamento de bibliotecas

from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import time
from six import StringIO
from IPython.display import Image  
import pydotplus
import import_ipynb
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
import itertools
from controllers.utils.utils import mostrar_dataframe_interativo
print("bibliotecas carregadas")


bibliotecas carregadas


In [2]:
#configuracoes e variaveis iniciais

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.6f}'.format
iterations_count = 10;
algoritms = ['knn', 'dt', 'svm', 'mlp']

print("configurações iniciais concluídas")


configurações iniciais concluídas


In [3]:
#carregamendo dos dados

df = pd.read_csv("data.csv")
print(f"dados carregados: {df['cid'].count()} registros")


dados carregados: 2139 registros


In [4]:
#ETL

df = df.drop(columns=['pidnum']) #pidnum é a coluna de identificação do paciente, não tem significado para o treinamento

# Identifica e remove colunas com valores iguais em todas as linhas
for col in df.columns:
    if df[col].nunique() == 1:        
        df = df.drop(columns=[col])
        print(f"coluna {col} excluida")
        
print("ETL concluido")

coluna zprior excluida
ETL concluido


In [5]:
# Embaralhando as linhas do DataFrame

df = df.sample(frac=1, random_state=None).reset_index(drop=True)
mostrar_dataframe_interativo(df)


IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [6]:
# Extratificação 

X = df.drop(columns=['cid'])  # Variáveis independentes (todas as colunas menos a 'cid')
y = df['cid']  # Variável dependente (alvo)
print("Variaveis independentes")
mostrar_dataframe_interativo(X)
print()
print("Variavel dependente (target)")
mostrar_dataframe_interativo(y)


Variaveis independentes


IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()


Variavel dependente (target)


IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [7]:
# Contagem das classes antes da divisão

print("Distribuição original das classes em 'cid':")
print(y.value_counts(normalize=True))  # Proporção das classes antes da divisão
print()


Distribuição original das classes em 'cid':
cid
0   0.756428
1   0.243572
Name: proportion, dtype: float64



In [8]:
# Primeiro, dividir em treino (50%) e restante (50% para validação + teste)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

print("dados splitados")

dados splitados


In [9]:
# Contagem das classes no conjunto de treino

print("Distribuição das classes no conjunto de treino (X_train):")
print(y_train.value_counts(normalize=True))  # Proporção das classes no treino
print()


Distribuição das classes no conjunto de treino (X_train):
cid
0   0.756782
1   0.243218
Name: proportion, dtype: float64



In [10]:
# Segunda divisão: validação (25%) e teste (25%)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("dados de validacao e teste splitados")


dados de validacao e teste splitados


In [11]:
# Contagem das classes nos conjuntos de validação e teste

print("Distribuição das classes no conjunto de validação (X_val):")
print(y_val.value_counts(normalize=True))  # Proporção das classes na validação
print()

print("Distribuição das classes no conjunto de teste (X_test):")
print(y_test.value_counts(normalize=True))  # Proporção das classes no teste
print()


Distribuição das classes no conjunto de validação (X_val):
cid
0   0.755140
1   0.244860
Name: proportion, dtype: float64

Distribuição das classes no conjunto de teste (X_test):
cid
0   0.757009
1   0.242991
Name: proportion, dtype: float64



In [12]:
#funcao para rodar o classificar e armazenar os resultados

def update_results(results,algoritm,iteration,y_val, classifier):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_val)  # Usando X_val e y_val para validação
    iteration['accuracy'] = metrics.accuracy_score(y_val, y_pred)
    iteration['confusion_matrix'] = metrics.confusion_matrix(y_val, y_pred).tolist()
    iteration['elapsed_time'] = time.time() - iteration['start_time']
    
    results[algoritm]['iterations'].append(iteration)

    # Atualizar melhores e piores índices e valores de acurácia e tempo
    if not results[algoritm]['best_accuracy'] or iteration['accuracy'] > results[algoritm]['best_accuracy']['accuracy']:
        results[algoritm]['best_accuracy'] = iteration

    if not results[algoritm]['worst_accuracy'] or iteration['accuracy'] < results[algoritm]['worst_accuracy']['accuracy']:
        results[algoritm]['worst_accuracy'] = iteration
        
    if not results['best_metrics']['best_accuracy'] or results['best_metrics']['best_accuracy']['accuracy'] < iteration['accuracy']: 
        results['best_metrics']['best_accuracy'] = dict(iteration)
        results['best_metrics']['best_accuracy']['algoritm'] = algoritm
    
    if not results['worst_metrics']['worst_accuracy'] or results['worst_metrics']['worst_accuracy']['accuracy'] > iteration['accuracy']: 
        results['worst_metrics']['worst_accuracy'] = dict(iteration)
        results['worst_metrics']['worst_accuracy']['algoritm'] = algoritm

    if not results[algoritm]['best_time'] or iteration['elapsed_time'] < results[algoritm]['best_time']['elapsed_time']:
        results[algoritm]['best_time'] = iteration

    if not results[algoritm]['worst_time'] or iteration['elapsed_time'] > results[algoritm]['worst_time']['elapsed_time']:
        results[algoritm]['worst_time'] = iteration
        
    if not results['best_metrics']['best_time'] or results['best_metrics']['best_time']['elapsed_time'] > iteration['elapsed_time']: 
        results['best_metrics']['best_time'] = dict(iteration)
        results['best_metrics']['best_time']['algoritm'] = algoritm
    
    if not results['worst_metrics']['worst_time'] or results['worst_metrics']['worst_time']['elapsed_time'] < iteration['elapsed_time']: 
        results['worst_metrics']['worst_time'] = dict(iteration)
        results['worst_metrics']['worst_time']['algoritm'] = algoritm

print("função de classificacao e atualização de resultados criada")

função de classificacao e atualização de resultados criada


In [13]:
# Dicionários para armazenar resultados de validação e teste

validation_results = {
    'iterations':0,
    'start_time':time.time(),
    'elapsed_time':0,
    'best_metrics': {'best_accuracy': None, 'best_time': None},
    'worst_metrics': {'worst_accuracy': None, 'worst_time': None}
}

test_results = {
    'iterations':0,
    'start_time':time.time(),
    'elapsed_time':0,
    'best_metrics': {'best_accuracy': None, 'best_time': None},
    'worst_metrics': {'worst_accuracy': None, 'worst_time': None}
}

for alg in algoritms:
    validation_results[alg] = {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None}
    test_results[alg] = {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None}

print("dicionario de resultados de validação e teste criado")

dicionario de resultados de validação e teste criado


In [22]:
#LOOP DE EXECUSSÕES DE VALIDAÇÃO (LOOP 1)

for i in range(iterations_count):
    
    ### KNN Classifier
    knn_iteration = {        
        'iteration':i,        
        'parameters':{
            'k':np.random.randint(1, 20), # Valor de K aleatório 
            'weights': np.random.choice(['uniform', 'distance']) # Pesos aleatórios
        },
        'start_time': time.time()
    }
    knn_clf = KNeighborsClassifier(n_neighbors=knn_iteration['parameters']['k'], weights=knn_iteration['parameters']['weights'])
    update_results(validation_results,'knn',knn_iteration,y_val,knn_clf)
    
    ### Decision Tree Classifier
    dt_iteration = {
        'iteration':i,        
        'parameters':{
            'max_depth': np.random.randint(1, 100),  # Aleatório entre 
            'min_samples_split': np.random.randint(2, 30),  # Aleatório entre 2 e 10 (influencia na poda)
            'criterion': np.random.choice(['gini', 'entropy']),  # Critério aleatório
            'ccp_alpha': np.random.uniform(0.0, 0.1)   # Parâmetro de complexidade de poda entre 0 e 0.05 (influencia na poda)
        },
        'start_time': time.time()
    }
    dt_clf = DecisionTreeClassifier(max_depth=dt_iteration['parameters']['max_depth'], min_samples_split=dt_iteration['parameters']['min_samples_split'], criterion=dt_iteration['parameters']['criterion'], ccp_alpha=dt_iteration['parameters']['ccp_alpha'])
    update_results(validation_results,'dt',dt_iteration,y_val,dt_clf)

    ### SVM Classifier
    svm_iteration = {
        'iteration':i,        
        'parameters':{
            'C': np.random.uniform(0.1, 1.0),  # Valor de erro aleatório (C)
            'kernel': np.random.choice(['rbf', 'poly'])  # Tipo de Kernel aleatório (linear excluido por ser mais demorado)
        },
        'start_time': time.time()
    }
    svm_clf = SVC(C=svm_iteration['parameters']['C'], kernel=svm_iteration['parameters']['kernel'])
    update_results(validation_results,'svm',svm_iteration,y_val,svm_clf) 

    ### MLP Classifier
    mlp_iteration = {
        'iteration':i,        
        'parameters':{
            'epochs': np.random.randint(50, 301),  # Número de épocas aleatório entre 50 e 300
            'learning_rate': np.random.uniform(0.001, 0.1),  # Taxa de aprendizagem aleatória
            'hidden_layers': np.random.randint(1, 6),  # Número de camadas escondidas aleatório entre 1 e 5
            'activation': np.random.choice(['identity', 'logistic', 'tanh', 'relu']),  # Função de ativação aleatória
            #'max_iter': 100,  # Aumenta o limite de iterações
            #'tol': 1e-3  # Tolerância um pouco maior para convergência
        },
        'start_time': time.time()
    }
    mlp_clf = MLPClassifier(hidden_layer_sizes=(mlp_iteration['parameters']['hidden_layers'],), max_iter=mlp_iteration['parameters']['epochs'], learning_rate_init=mlp_iteration['parameters']['learning_rate'], activation=mlp_iteration['parameters']['activation'])
    update_results(validation_results,'mlp',mlp_iteration,y_val,mlp_clf) 

    validation_results['iterations'] += 1

validation_results['elapsed_time'] = time.time() - validation_results['start_time']

print("loop de validacao concluido")


loop de validacao concluido


In [23]:
# Exibir resultados de validação
print("mostrar json ? (descomente)")
print(json.dumps(validation_results, indent=4))

mostrar json ? (descomente)
{
    "iterations": 30,
    "start_time": 1729972015.0800989,
    "elapsed_time": 64.83885049819946,
    "best_metrics": {
        "best_accuracy": {
            "iteration": 4,
            "parameters": {
                "max_depth": 74,
                "min_samples_split": 26,
                "criterion": "entropy",
                "ccp_alpha": 0.007050382755709017
            },
            "start_time": 1729972078.563963,
            "accuracy": 0.902803738317757,
            "confusion_matrix": [
                [
                    387,
                    17
                ],
                [
                    35,
                    96
                ]
            ],
            "elapsed_time": 0.01585865020751953,
            "algoritm": "dt"
        },
        "best_time": {
            "iteration": 2,
            "parameters": {
                "max_depth": 41,
                "min_samples_split": 8,
                "criterion": "gini",
    

In [24]:
#tabulação dos resultados da validação e calculo das medias

validation_accuracy_table = {
    'repetição':[]
}
validation_time_table = {
    'repetição':[]
}
validation_accuracy_avgs = {}
validation_time_avgs = {}

for alg in algoritms:
    validation_accuracy_table[alg] = []
    validation_accuracy_avgs[alg] = 0
    validation_time_table[alg] = []
    validation_time_avgs[alg] = 0

for i in range(iterations_count):
    validation_accuracy_table['repetição'].append(i+1)  
    validation_time_table['repetição'].append(i+1)  
    for alg in algoritms:
        if len(validation_results[alg]['iterations']) > 0:
            validation_accuracy_table[alg].append(validation_results[alg]['iterations'][i]['accuracy'])
            validation_accuracy_avgs[alg] += validation_results[alg]['iterations'][i]['accuracy']
            validation_time_table[alg].append(validation_results[alg]['iterations'][i]['elapsed_time'])
            validation_time_avgs[alg] += validation_results[alg]['iterations'][i]['elapsed_time']
        else:
            validation_accuracy_table[alg].append(0)
            validation_time_table[alg].append(float('inf'))

validation_accuracy_table['repetição'].append('AVG')
validation_accuracy_table['repetição'].append('WORST')
validation_accuracy_table['repetição'].append('BEST')
validation_time_table['repetição'].append('AVG')
validation_time_table['repetição'].append('WORST')
validation_time_table['repetição'].append('BEST')
for alg in algoritms:
    validation_accuracy_avgs[alg] = validation_accuracy_avgs[alg] / iterations_count
    validation_time_avgs[alg] = validation_time_avgs[alg] / iterations_count
    validation_accuracy_table[alg].append(validation_accuracy_avgs[alg])
    validation_accuracy_table[alg].append(validation_results[alg]['worst_accuracy']['accuracy'])
    validation_accuracy_table[alg].append(validation_results[alg]['best_accuracy']['accuracy'])
    validation_time_table[alg].append(validation_time_avgs[alg])
    validation_time_table[alg].append(validation_results[alg]['worst_time']['elapsed_time'])
    validation_time_table[alg].append(validation_results[alg]['best_time']['elapsed_time'])




validation_accuracy_table = pd.DataFrame(validation_accuracy_table)
validation_time_table = pd.DataFrame(validation_time_table)



print()
print("Medias de acuracias na validação")
print(validation_accuracy_table)
#mostrar_dataframe_interativo(validation_accuracy_table,min(len(validation_accuracy_table),iterations_count)+1,min(len(validation_accuracy_table),iterations_count)+1)
print()
print("Medias de tempos na validação")
print(validation_time_table)
#mostrar_dataframe_interativo(validation_time_table,min(len(validation_time_table),iterations_count)+1,min(len(validation_time_table),iterations_count)+1)


Medias de acuracias na validação
   repetição      knn       dt      svm      mlp
0          1 0.844860 0.865421 0.839252 0.755140
1          2 0.841121 0.857944 0.867290 0.755140
2          3 0.854206 0.865421 0.863551 0.822430
3          4 0.841121 0.857944 0.805607 0.813084
4          5 0.856075 0.857944 0.852336 0.852336
5          6 0.811215 0.857944 0.837383 0.768224
6          7 0.844860 0.857944 0.857944 0.755140
7          8 0.835514 0.857944 0.850467 0.755140
8          9 0.856075 0.857944 0.826168 0.798131
9         10 0.841121 0.857944 0.852336 0.755140
10       AVG 0.842617 0.859439 0.845234 0.782991
11     WORST 0.811215 0.857944 0.803738 0.753271
12      BEST 0.856075 0.902804 0.867290 0.854206

Medias de tempos na validação
   repetição      knn       dt      svm      mlp
0          1 0.133900 0.008647 0.075114 0.034854
1          2 0.094068 0.013309 0.086449 0.026165
2          3 0.059687 0.000000 0.092410 0.109370
3          4 0.081368 0.015240 0.044703 0.077136
4   

In [26]:
#LOOP DE EXECUSSÕES DE TESTE (LOOP 2)

for i in range(iterations_count):
    
    ### KNN Classifier
    knn_iteration = {        
        'iteration':i,        
        'parameters':validation_results['knn']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    knn_clf = KNeighborsClassifier(n_neighbors=knn_iteration['parameters']['k'], weights=knn_iteration['parameters']['weights'])
    update_results(test_results,'knn',knn_iteration,y_val,knn_clf)
    
    ### Decision Tree Classifier
    dt_iteration = {
        'iteration':i,        
        'parameters':validation_results['dt']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    dt_clf = DecisionTreeClassifier(max_depth=dt_iteration['parameters']['max_depth'], min_samples_split=dt_iteration['parameters']['min_samples_split'], criterion=dt_iteration['parameters']['criterion'], ccp_alpha=dt_iteration['parameters']['ccp_alpha'])
    update_results(test_results,'dt',dt_iteration,y_val,dt_clf)

    ### SVM Classifier
    svm_iteration = {
        'iteration':i,        
        'parameters':validation_results['svm']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    svm_clf = SVC(C=svm_iteration['parameters']['C'], kernel=svm_iteration['parameters']['kernel'])
    update_results(test_results,'svm',svm_iteration,y_val,svm_clf)

    ### MLP Classifier
    mlp_iteration = {
        'iteration':i,        
        'parameters':validation_results['mlp']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    mlp_clf = MLPClassifier(hidden_layer_sizes=(mlp_iteration['parameters']['hidden_layers'],), max_iter=mlp_iteration['parameters']['epochs'], learning_rate_init=mlp_iteration['parameters']['learning_rate'], activation=mlp_iteration['parameters']['activation'])
    update_results(test_results,'mlp',mlp_iteration,y_val,mlp_clf) 

    test_results['iterations'] += 1

test_results['elapsed_time'] = time.time() - test_results['start_time']

print("loop de testes concluido")

loop de testes concluido


In [28]:
# Exibir resultados de teste
print("mostrar json ? (descomente)")
print(json.dumps(test_results, indent=4))

mostrar json ? (descomente)
{
    "iterations": 20,
    "start_time": 1729972015.0800989,
    "elapsed_time": 216.69672966003418,
    "best_metrics": {
        "best_accuracy": {
            "iteration": 0,
            "parameters": {
                "max_depth": 74,
                "min_samples_split": 26,
                "criterion": "entropy",
                "ccp_alpha": 0.007050382755709017
            },
            "start_time": 1729972228.1803713,
            "accuracy": 0.902803738317757,
            "confusion_matrix": [
                [
                    387,
                    17
                ],
                [
                    35,
                    96
                ]
            ],
            "elapsed_time": 0.014989852905273438,
            "algoritm": "dt"
        },
        "best_time": {
            "iteration": 9,
            "parameters": {
                "max_depth": 74,
                "min_samples_split": 11,
                "criterion": "entropy

In [31]:
#tabulação dos resultados da testes e calculo das medias

test_accuracy_table = {
    'repetição':[]
}
test_time_table = {
    'repetição':[]
}
test_accuracy_avgs = {}
test_time_avgs = {}

for alg in algoritms:
    test_accuracy_table[alg] = []
    test_accuracy_avgs[alg] = 0
    test_time_table[alg] = []
    test_time_avgs[alg] = 0

for i in range(iterations_count):
    test_accuracy_table['repetição'].append(i+1)  
    test_time_table['repetição'].append(i+1)  
    for alg in algoritms:
        if len(test_results[alg]['iterations']) > 0:
            test_accuracy_table[alg].append(test_results[alg]['iterations'][i]['accuracy'])
            test_accuracy_avgs[alg] += test_results[alg]['iterations'][i]['accuracy']
            test_time_table[alg].append(test_results[alg]['iterations'][i]['elapsed_time'])
            test_time_avgs[alg] += test_results[alg]['iterations'][i]['elapsed_time']
        else:
            test_accuracy_table[alg].append(0)
            test_time_table[alg].append(float('inf'))

test_accuracy_table['repetição'].append('AVG')
test_accuracy_table['repetição'].append('WORST')
test_accuracy_table['repetição'].append('BEST')
test_time_table['repetição'].append('AVG')
test_time_table['repetição'].append('WORST')
test_time_table['repetição'].append('BEST')
for alg in algoritms:
    test_accuracy_avgs[alg] = test_accuracy_avgs[alg] / iterations_count
    test_time_avgs[alg] = test_time_avgs[alg] / iterations_count
    test_accuracy_table[alg].append(test_accuracy_avgs[alg])
    test_accuracy_table[alg].append(test_results[alg]['worst_accuracy']['accuracy'])
    test_accuracy_table[alg].append(test_results[alg]['best_accuracy']['accuracy'])
    test_time_table[alg].append(test_time_avgs[alg])
    test_time_table[alg].append(test_results[alg]['worst_time']['elapsed_time'])
    test_time_table[alg].append(test_results[alg]['best_time']['elapsed_time'])




test_accuracy_table = pd.DataFrame(test_accuracy_table)
test_time_table = pd.DataFrame(test_time_table)



print()
print("Medias de acuracias no teste")
print(test_accuracy_table)
#mostrar_dataframe_interativo(test_accuracy_table,min(len(test_accuracy_table),iterations_count)+1,min(len(test_accuracy_table),iterations_count)+1)
print()
print("Medias de tempos no teste")
print(test_time_table)
#mostrar_dataframe_interativo(test_time_table,min(len(test_time_table),iterations_count)+1,min(len(test_time_table),iterations_count)+1)


Medias de acuracias no teste
   repetição      knn       dt      svm      mlp
0          1 0.856075 0.865421 0.867290 0.839252
1          2 0.856075 0.865421 0.867290 0.826168
2          3 0.856075 0.865421 0.867290 0.837383
3          4 0.856075 0.865421 0.867290 0.859813
4          5 0.856075 0.865421 0.867290 0.828037
5          6 0.856075 0.865421 0.867290 0.856075
6          7 0.856075 0.865421 0.867290 0.857944
7          8 0.856075 0.865421 0.867290 0.835514
8          9 0.856075 0.865421 0.867290 0.801869
9         10 0.856075 0.865421 0.867290 0.833645
10       AVG 0.856075 0.865421 0.867290 0.837570
11     WORST 0.856075 0.865421 0.867290 0.801869
12      BEST 0.856075 0.902804 0.867290 0.859813

Medias de tempos no teste
   repetição      knn       dt      svm      mlp
0          1 0.093656 0.033460 0.093713 0.118008
1          2 0.097633 0.018937 0.108091 0.126373
2          3 0.106530 0.026126 0.102317 0.160737
3          4 0.094546 0.017725 0.127033 0.156732
4          5

In [37]:
#Aplicando estatistica de Kruskal-Wallis
df_statics = test_accuracy_table.iloc[:-3] #elimina as 3 ultimas linhas, que contem as medias, melhores e piores

kruskal_result = kruskal(df_statics['knn'], df_statics['dt'], df_statics['svm'], df_statics['mlp'])
print("Resultado do teste de Kruskal-Wallis:")
print("Estatística:", kruskal_result.statistic)
print("p-valor:", kruskal_result.pvalue)

Resultado do teste de Kruskal-Wallis:
Estatística: 35.682492581602375
p-valor: 8.73995494271708e-08


In [38]:
# # Comparação par-a-par com Mann-Whitney

classifiers = {
    'MLP': df_statics['mlp'],
    'DT': df_statics['dt'],
    'SVM': df_statics['svm'],
    'KNN': df_statics['knn']
}

print("\nTeste de Mann-Whitney (comparação dois a dois):")
for (name1, acc1), (name2, acc2) in itertools.combinations(classifiers.items(), 2):
    mannwhitney_result = mannwhitneyu(acc1, acc2, alternative='two-sided')
    print(f"Comparação {name1} vs {name2}:")
    print("  Estatística:", mannwhitney_result.statistic)
    print("  p-valor:", mannwhitney_result.pvalue)


Teste de Mann-Whitney (comparação dois a dois):
Comparação MLP vs DT:
  Estatística: 0.0
  p-valor: 6.386444750436982e-05
Comparação MLP vs SVM:
  Estatística: 0.0
  p-valor: 6.386444750436982e-05
Comparação MLP vs KNN:
  Estatística: 25.0
  p-valor: 0.04263490442783271
Comparação DT vs SVM:
  Estatística: 0.0
  p-valor: 1.5937911688066244e-05
Comparação DT vs KNN:
  Estatística: 100.0
  p-valor: 1.5937911688066244e-05
Comparação SVM vs KNN:
  Estatística: 100.0
  p-valor: 1.5937911688066244e-05
